In [ ]:
import os
import stumpy
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt

In [ ]:
def get_train_test_data(data_folder: str, selected_folder: str, label_filename: str):
    files = [f.split(".")[0] for f in os.listdir(f"{data_folder}/{selected_folder}")]
    labels = pd.read_csv(
        f"{data_folder}/{selected_folder}/{label_filename}.csv", index_col=0
    )
    train_files = [file for file in files if file != label_filename]

    return train_files, labels

In [ ]:
def from_files_to_anomaly_type(train_files: list) -> dict:
    from_files_to_anomaly_type = {}

    for file in train_files:
        if file.startswith("1"):
            from_files_to_anomaly_type[file] = "bursty input"
        elif file.startswith("2"):
            from_files_to_anomaly_type[file] = "stalled input"
        elif file.startswith("3"):
            from_files_to_anomaly_type[file] = "CPU contention"
        else:
            raise ValueError(f"Unknown file {file}.")

    return from_files_to_anomaly_type

In [ ]:
def split_anomalies_and_references(
    train_files: list,
    labels: pd.DataFrame,
    data_folder: str,
    selected_folder: str,
    from_files_to_anomaly_type: dict,
) -> (dict, dict):
    references = {}
    anomalies = {}

    for filename in train_files:
        train_file = pd.read_csv(
            f"{data_folder}/{selected_folder}/{filename}.csv", index_col=0
        )
        train_file["original_filename"] = filename
        train_file["timestamp"] = train_file.index

        label_file = labels.loc[labels["trace_id"] == filename, :]

        for i in label_file.index:
            ano_id = label_file.loc[i, "ano_id"]
            selection_ref = train_file.loc[
                (train_file["timestamp"] >= label_file["ref_start"][i])
                & (train_file["timestamp"] < label_file["ref_end"][i]),
                :,
            ].copy()
            selection_ref["ano_id"] = ano_id
            selection_ref["type_data"] = 0
            selection_ano = train_file.loc[
                (train_file["timestamp"] >= label_file["ano_start"][i])
                & (train_file["timestamp"] <= label_file["ano_end"][i]),
                :,
            ].copy()
            selection_ano["ano_id"] = ano_id
            selection_ano["type_data"] = 1
            references[
                f"{from_files_to_anomaly_type[filename]}_{filename}_{i}"
            ] = selection_ref
            anomalies[
                f"{from_files_to_anomaly_type[filename]}_{filename}_{i}"
            ] = selection_ano

    assert references.keys() == anomalies.keys()
    references = pd.concat(references).droplevel(1)
    anomalies = pd.concat(anomalies).droplevel(1)

    return references, anomalies

In [ ]:
def remove_correlated_features(df, correlation_threshold=0.9):
    # Step 1: Calculate the correlation matrix
    correlation_matrix = df.corr()

    # Step 2: Create a graph based on pairwise correlations
    G = nx.Graph()

    # Add nodes (features) to the graph
    G.add_nodes_from(correlation_matrix.columns)

    # Add edges between nodes if the correlation exceeds a threshold
    for i in range(len(correlation_matrix.columns[:-4])):  # Last 4 columns are metadata
        for j in range(i):
            if abs(correlation_matrix.iloc[i, j]) > correlation_threshold:
                G.add_edge(correlation_matrix.columns[i], correlation_matrix.columns[j])

    # Step 3: Extract clusters from the graph
    clusters = list(nx.connected_components(G))

    # Step 4: Select one representative feature from each cluster
    selected_features = [cluster.pop() for cluster in clusters]

    return selected_features

In [ ]:
def false_positive_filtering(refs, anos, cols):
    new_cols = []
    refs_df = refs[cols]
    anos_df = anos[cols]
    cols_to_visit = list(anos_df.columns[:-4])
    for ano in anos.index.unique():
        cols_for_this_ano = []
        nb_matches = []
        for col in cols_to_visit:
            pattern = anos_df.loc[ano, col]
            ts = refs_df.loc[:, col]
            matches = stumpy.match(pattern, ts, max_distance=28.0)
            nb_matches.append(len(matches))
            if len(list(matches)) <= 1:
                if col not in new_cols:
                    cols_for_this_ano.append(col)
            # else:
            # print(f"Found {len(matches)} match(es) for {col} in ano {ano}")
        if not cols_for_this_ano:
            new_cols.append(cols_to_visit[np.array(nb_matches).argmin()])
        else:
            new_cols.append(cols_for_this_ano)
    return new_cols

In [ ]:
def assign_cols_per_ano(anos, new_filtered_features):
    anos["filtered_columns"] = None
    for i, ano in enumerate(anos.index.unique()):
        anos.loc[ano, "filtered_columns"] = str(new_filtered_features[i])
    anos["filtered_columns"] = anos["filtered_columns"].apply(
        lambda x: x.strip("][").split(", ")
    )
    return anos

In [ ]:
def class_entropy(nb_ts_a: list, nb_ts_r: list) -> float:
    """
    Calculate the class entropy of a feature, which is the information
    needed to describe the class distributions between two time serie.

    Parameters
    ----------
    nb_ts_a : list
        Number of observations inside a time series belonging to the abnormal
        class.
    nb_ts_r : list
        Number of observations inside a time series belonging to the reference
        class.

    Returns
    -------
    float
        The class entropy.
    """

    if nb_ts_a == 0 or nb_ts_r == 0:
        raise ValueError(
            f"One of the time series is empty. Len of TSA is {nb_ts_a} and len of TSR"
            f" is {nb_ts_r}."
        )
    p_a = nb_ts_a / (nb_ts_a + nb_ts_r)
    p_r = nb_ts_r / (nb_ts_a + nb_ts_r)
    h_class = p_a * np.log2(1 / p_a) + p_r * np.log2(1 / p_r)

    return h_class

In [ ]:
def shuffle_observations_if_duplicates(
    sorted_values: pd.DataFrame, feature
) -> pd.DataFrame:
    """
    Shuffle the observations if there are duplicates in the sorted values.

    Parameters
    ----------
    sorted_values : pd.DataFrame
        The sorted values.

    Returns
    -------
    pd.DataFrame
        The sorted values with shuffled values for duplicates.
    """

    # On récupère le nombre de références et d'anomalies pour chaque modalité
    value_type_to_count = sorted_values.groupby(feature).value_counts().to_dict()

    # On récupère le nombre de valeurs distinctes pour chaque modalité (soit 1 lorsque
    # pas de doublons, soit 2, lorsqu'il y a des doublons)
    value_to_count_distinct = (
        sorted_values.drop_duplicates().groupby(feature).count().to_dict()["type_data"]
    )

    # On récupère les modalités distinctes (les différentes valeurs prises par la
    # feature)
    modalities = set(sorted_values[feature].tolist())
    # En fait, ce qui est un peu bizarre c'est qu'on a des valeurs continues, mais là
    # on va les traiter comme des valeurs discrètes :
    # Par exemple si on considère une colonne qui prend les valeurs 501.03, 501.03,
    # 502.4, 502.4, 505.0, on itère sur 501.03, 502.4, 505.0

    # On parcourt chaque modalité
    for modality in modalities:
        # On récupère le premier type de données observé (ano ou ref, donc 1 ou 0) (ça
        # nous sera utile plus tard)
        last_type_data = sorted_values.loc[
            sorted_values[feature] == modality, "type_data"
        ].tolist()[0]

        # Cas où il n'y a pas de doublons
        if value_to_count_distinct[modality] == 1:
            # On ne fait rien
            continue

        # Cas où il y a des doublons
        else:
            # On va shuffle dans le pire ordre possible

            # D'abord on instancie les variables nécessaires
            list_values = []
            nb_refs = value_type_to_count[(modality, 0)]
            nb_anos = value_type_to_count[(modality, 1)]
            nb_total = nb_refs + nb_anos
            # Hop maintenant c'est parti pour le shuffle

            # Cas où il n'y a pas le même nombre de références et d'anomalie
            #  (cas le plus chiant)
            if nb_refs != nb_anos:
                # On instancie de nouveau des variables nécessaires
                biggest = int(
                    nb_refs < nb_anos
                )  # 1 si on a plus d'anomalies que de références, 0 sinon
                smallest = int(
                    nb_refs > nb_anos
                )  # 1 si on a plus de références que d'anomalies, 0 sinon
                nb_smallest = min(
                    nb_refs, nb_anos
                )  # Nombre de fois où on va mettre la valeur la moins représentée

                # On commence par mettre la valeur la plus représentée partout
                list_values = [biggest] * nb_total

                # Puis on cherche si le dernier type de donnée observé est le plus
                # représenté ou le moins représenté pour savoir où commencer
                start_smallest = 0 if smallest != last_type_data else 1

                # On parcourt la liste 2 par 2 pour mettre la valeur la moins
                # représentée
                for i in range(start_smallest, nb_smallest * 2, 2):
                    list_values[i] = smallest

            # Cas où il y a le même nombre de références et d'anomalies
            # (cas le plus simple)
            else:
                # On parcourt le nombre total d'observations
                for i in range(nb_total):
                    # On alterne entre 0 et 1 en commençant par la valeur opposée à la
                    # dernière valeur observée
                    list_values.append(abs(last_type_data - i % 2 - 1))

            # On récupère le dernier type de donnée observé (toujours 1 ou 0)
            last_type_data = sorted_values.loc[
                sorted_values[feature] == modality, "type_data"
            ].tolist()[-1]

        # On vérifie que la longueur de la liste est bien égale au nombre
        # d'observations pour la modalité
        assert (
            len(list_values)
            == sorted_values[sorted_values[feature] == modality].shape[0]
        ), (
            f"Len of list_values {len(list_values)} is not equal to the number of"
            " observations for the modality"
            f" {sorted_values[sorted_values[feature]==modality].shape[0]}."
        )
        # On met à jour le type de donnée pour la modalité
        sorted_values.loc[sorted_values[feature] == modality, "type_data"] = list_values

    return sorted_values

In [ ]:
def segmentation_entropy(shuffled_values: pd.DataFrame) -> float:
    """
    Calculate the segmentation entropy of a feature, which is the information
    needed to describe how merged points are segmented by class labels.

    Parameters:
    shuffled_values (pd.DataFrame): A DataFrame containing the shuffled values.

    Returns:
    float: The segmentation entropy of the feature.
    """
    # On récupère la time serie
    ts = shuffled_values["type_data"].tolist()

    # Stocke la première valeur de la liste
    past_value = ts[0]

    # Liste pour stocker les valeurs à l'intérieur d'un segment
    values_inside_segment = []

    # Variable pour stocker la segmentation entropy
    segmentation_ent = 0.0

    # Parcourt chaque valeur dans la time serie
    for value in ts:
        # Si la valeur est différente de la valeur précédente
        if value != past_value:
            # On a un nouveau segment, il faut calculer l'entropie de segmentation
            # partielle du précédent segment
            pi = len(values_inside_segment) / shuffled_values.shape[0]
            segmentation_ent += pi * np.log(1 / pi)

            # On réinitialise la liste des valeurs à l'intérieur du segment avec la
            # nouvelle valeur
            values_inside_segment = [value]
        else:
            # On stocke les valeurs à l'intérieur du segment tant qu'un nouveau segment
            # n'est pas créé
            values_inside_segment.append(value)

        # On met à jour la valeur précédente avec la valeur actuelle
        past_value = value

    return segmentation_ent

In [ ]:
def entropy_based_single_feature_reward(refs: pd.DataFrame, anos: pd.DataFrame) -> dict:
    """
    Calculates the reward function for a single feature based on the reference
    data and the annotated data.

    Parameters:
    refs (pandas.DataFrame): The reference data.
    anos (pandas.DataFrame): The abnormal data.

    Returns:
    dict: A dictionary containing the single reward function for each feature.
    """

    distances = {}
    # On calcule la class entropy
    class_ent = class_entropy(refs.shape[0], anos.shape[0])
    # On calcule la segmentation entropy pour chaque feature sauf type_data
    for feature in [col for col in refs.columns if col != "type_data"]:
        # On concatène les références et les anomalies pour la feature
        all_values = pd.concat(
            [refs[[feature, "type_data"]], anos[[feature, "type_data"]]]
        )
        # On trie les valeurs par feature puis par type_data
        sorted_values = all_values.sort_values(by=[feature, "type_data"])
        # On shuffle les valeurs si on a des doublons
        shuffled_values = shuffle_observations_if_duplicates(sorted_values, feature)
        # On calcule la segmentation entropy
        segmentation_ent = segmentation_entropy(shuffled_values)
        # On calcule la single reward function
        distance = class_ent / segmentation_ent
        # On stocke la single reward function dans le dictionnaire
        distances[feature] = distance

        # if all(value < 0 for value in distances.values()):
        #    positive_distances = {
        #        key: np.abs(value) for key, value in distances.items()
        #    }

        sorted_distances = dict(
            sorted(distances.items(), key=lambda x: x[1], reverse=True)
        )

    return sorted_distances

In [ ]:
def maximum_leap(distances: dict) -> float:
    """
    Calculate the maximum leap between the distances of the neighboring ranked
    features.

    Parameters
    ----------
    distances : dict
        The ranking of all remaining features based on their individual reward.

    Returns
    -------
    maximum_leap : float
        The maximum leap.

    """

    leaps = [
        last_distance - distance
        for last_distance, distance in zip(
            distances.values(), list(distances.values())[1:]
        )
    ]

    maximum_leap = max(leaps)

    return maximum_leap

In [ ]:
def reward_leap_filter(distances: dict) -> list:
    """
    Discard the features that rank below a sharp drop in the reward.

    Parameters
    ----------
    distances : dict
        The ranking of all remaining features based on their individual reward.

    Returns
    -------
    filtered_features : dict
        The remaining features after filtering.

    """
    if len(distances) > 0:  # j'ai ajouté ça au cas où
        threshold = maximum_leap(distances)
        to_keep = set()
        last_distance = 0

        for feature, distance in distances.items():
            if last_distance != 0:
                leap = last_distance - distance
                if leap == threshold:
                    break
            last_distance = distance
            to_keep.update([feature])

        filtered_features = [
            feature for feature in distances.keys() if feature in to_keep
        ]

        return filtered_features

    else:
        return None

In [ ]:
DATA_FOLDER = "data"
FOLDERS = ["folder_1", "folder_2"]
SEL_FOLDER = FOLDERS[0]
train_files, labels = get_train_test_data(DATA_FOLDER, SEL_FOLDER, "labels")
from_files_to_anomaly_type = from_files_to_anomaly_type(train_files)
refs, anos = split_anomalies_and_references(
    train_files, labels, DATA_FOLDER, SEL_FOLDER, from_files_to_anomaly_type
)
all_data = pd.concat([refs, anos])

In [ ]:
all_data.head()

In [ ]:
# STEP 1
filtered_features = remove_correlated_features(all_data)

In [ ]:
filtered_features

In [ ]:
anos

In [ ]:
# STEP2
new_filtered_features = false_positive_filtering(refs, anos, filtered_features)
new_anos = assign_cols_per_ano(anos, new_filtered_features)

In [ ]:
new_anos

In [ ]:
# distances = []

# for ano_index in new_anos.index.unique():
#     ano = new_anos.loc[ano_index]
#     columns_list = ano["filtered_columns"].values
#     print(ano_index)
#     for columns in np.unique(columns_list):
#         columns = [s.replace("'", "") for s in columns]
#         filtered_ano = ano[columns + ["type_data"]]
#         ref = refs[columns + ["type_data"]]
#         print(filtered_ano.shape)
#         distance = entropy_based_single_feature_reward(ref, filtered_ano)
#         print(distance)
#         distances.append(distance)

In [ ]:
bursty_refs = refs[refs.index.str.startswith("bursty")]
bursty_anos = new_anos[anos.index.str.startswith("bursty")]
stalled_refs = refs[refs.index.str.startswith("stalled")]
stalled_anos = new_anos[anos.index.str.startswith("stalled")]
cpu_refs = refs[refs.index.str.startswith("CPU")]
cpu_anos = new_anos[anos.index.str.startswith("CPU")]

In [ ]:
# STEP 3

distances_bursty = entropy_based_single_feature_reward(
    bursty_refs,
    bursty_anos,
)

distances_stalled = entropy_based_single_feature_reward(
    stalled_refs,
    stalled_anos,
)

distances_cpu = entropy_based_single_feature_reward(
    cpu_refs,
    cpu_anos,
)

In [ ]:
distances_bursty

In [ ]:
distances_stalled

In [ ]:
distances_cpu

In [ ]:
def get_explanatory_features(anos: pd.DataFrame, distances: dict) -> dict:
    selected_features = {}

    for ano_index in anos.index.unique():
        ano = anos.loc[ano_index]
        filtered_cols = ano["filtered_columns"].values
        for cols in np.unique(filtered_cols):
            cols = [s.replace("'", "") for s in cols]
            selected_distances = {
                feat: dist for feat, dist in distances.items() if feat in cols
            }
            if len(selected_distances) > 1:
                filtered_features = reward_leap_filter(selected_distances)
                print(filtered_features)
                selected_features[ano_index] = filtered_features
            else:
                selected_features[ano_index] = list(selected_distances.keys())

    return selected_features

In [ ]:
selected_features_bursty = get_explanatory_features(bursty_anos, distances_bursty)
selected_features_bursty

In [ ]:
selected_features_stalled = get_explanatory_features(stalled_anos, distances_stalled)
selected_features_stalled

In [ ]:
selected_features_cpu = get_explanatory_features(cpu_anos, distances_cpu)
selected_features_cpu

In [ ]:
for ano_index in stalled_anos.index.unique():
    stalled_ano = stalled_anos.loc[ano_index]
    columns_list = stalled_ano["filtered_columns"].values
    print(ano_index)
    for i, columns in enumerate(np.unique(columns_list)):
        fig, ax = plt.subplots(1, 1)
        columns = [s.replace("'", "") for s in columns]
        selected_distances_stalled = {
            feat: dist for feat, dist in distances_stalled.items() if feat in columns
        }
        print(selected_distances_stalled)
        print(maximum_leap(selected_distances_stalled))
        ax.bar(selected_distances_stalled.keys(), selected_distances_stalled.values())
        ax.axhline(y=np.median(list(selected_distances_stalled.values())), c="r")

plt.show()

In [ ]:
for ano_index in bursty_anos.index.unique():
    bursty_ano = bursty_anos.loc[ano_index]
    columns_list = bursty_ano["filtered_columns"].values
    # print(ano_index)
    for i, columns in enumerate(np.unique(columns_list)):
        columns = [s.replace("'", "") for s in columns]
        selected_distances_bursty = {
            feat: dist for feat, dist in distances_bursty.items() if feat in columns
        }
        # print(selected_distances_bursty)
        selected_features_bursty = reward_leap_filter(selected_distances_bursty)
        # print(selected_features_bursty)

        fig, ax = plt.subplots(1, 1)

        features_names = list(selected_distances_bursty.keys())

        all_distances = list(selected_distances_bursty.values())

        colors = [
            "orange" if feature in selected_features_bursty else "b"
            for feature in features_names
        ]

        features_names = [
            "..." + feature[-20:] if len(feature) > 10 else feature
            for feature in features_names
        ]

        ax.bar(features_names, all_distances, color=colors)
        ax.set_xticklabels(features_names, rotation=90)
        ax.set_title(f"Anomaly {ano_index}")
        # ax.axhline(y=np.median(list(selected_distances_bursty.values())), c="r")

plt.show()

In [ ]:
for ano_index in cpu_anos.index.unique():
    cpu_ano = cpu_anos.loc[ano_index]
    columns_list = cpu_ano["filtered_columns"].values
    # print(ano_index)
    for i, columns in enumerate(np.unique(columns_list)):
        try:
            columns = [s.replace("'", "") for s in columns]
            selected_distances_cpu = {
                feat: dist for feat, dist in distances_cpu.items() if feat in columns
            }
            # print(selected_distances_cpu)
            selected_features_cpu = reward_leap_filter(selected_distances_cpu)
            # print(selected_features_cpu)

            fig, ax = plt.subplots(1, 1)

            features_names = list(selected_distances_cpu.keys())

            all_distances = list(selected_distances_cpu.values())

            colors = [
                "orange" if feature in selected_features_cpu else "b"
                for feature in features_names
            ]

            features_names = [
                "..." + feature[-20:] if len(feature) > 10 else feature
                for feature in features_names
            ]

            ax.bar(features_names, all_distances, color=colors)
            ax.set_xticklabels(features_names, rotation=90)
            ax.set_title(f"Anomaly {ano_index}")
            # ax.axhline(y=np.median(list(selected_distances_cpu.values())), c="r")
        except:
            pass

plt.show()

In [ ]:
for ano_index in stalled_anos.index.unique():
    stalled_ano = stalled_anos.loc[ano_index]
    columns_list = stalled_ano["filtered_columns"].values
    # print(ano_index)
    for i, columns in enumerate(np.unique(columns_list)):
        columns = [s.replace("'", "") for s in columns]
        selected_distances_stalled = {
            feat: dist for feat, dist in distances_stalled.items() if feat in columns
        }
        # print(selected_distances_stalled)
        selected_features_stalled = reward_leap_filter(selected_distances_stalled)
        # print(selected_features_stalled)

        fig, ax = plt.subplots(1, 1)

        features_names = list(selected_distances_stalled.keys())

        all_distances = list(selected_distances_stalled.values())

        colors = [
            "orange" if feature in selected_features_stalled else "b"
            for feature in features_names
        ]

        features_names = [
            "..." + feature[-20:] if len(feature) > 10 else feature
            for feature in features_names
        ]

        ax.bar(features_names, all_distances, color=colors)
        ax.set_xticklabels(features_names, rotation=90)
        ax.set_title(f"Anomaly {ano_index}")
        # ax.axhline(y=np.median(list(selected_distances_stalled.values())), c="r")

plt.show()

In [ ]:
selected_features_bursty, selected_features_stalled, selected_features_cpu

In [ ]:
explanatory_features = {
    **selected_features_bursty,
    **selected_features_stalled,
    **selected_features_cpu,
}

In [ ]:
len(explanatory_features)

In [ ]:
explanatory_features_df = pd.DataFrame(
    list(explanatory_features.items()), columns=["index", "explanation"]
)

In [ ]:
explanatory_features_df

In [ ]:
labels_df = labels[["trace_id", "ano_id"]].copy()

In [ ]:
explanations = pd.merge(
    labels_df, explanatory_features_df, left_index=True, right_index=True
)

In [ ]:
explanations.drop(["index"], axis=1, inplace=True)

In [ ]:
explanations

In [ ]:
def get_features_integer_indice(features: list, anomalies: pd.DataFrame):
    """
    Returns the indices of the specified features in the anomalies DataFrame.

    Parameters:
    - features (list): A list of feature names.
    - anomalies (pd.DataFrame): A DataFrame containing the anomalies.

    Returns:
    - indices (list): A list of integer indices corresponding to the features in the
    anomalies DataFrame.
    """
    indices = []
    for feature in features:
        indice = anomalies.columns.get_loc(feature)
        indices.append(indice)

    return indices

In [ ]:
explanations["explanation"] = explanations["explanation"].apply(
    lambda x: get_features_integer_indice(x, anos)
)

In [ ]:
explanations

In [ ]:
explanations["exp_size"] = explanations["explanation"].apply(lambda x: len(x))

In [ ]:
explanations

In [ ]:
import numpy as np

In [ ]:
explanations = [
    [
        "driver_StreamingMetrics_streaming_lastCompletedBatch_schedulingDelay_value",
        "driver_StreamingMetrics_streaming_lastReceivedBatch_records_value",
    ],
    [
        "driver_StreamingMetrics_streaming_lastCompletedBatch_schedulingDelay_value",
        "driver_StreamingMetrics_streaming_lastReceivedBatch_records_value",
    ],
    [
        "driver_StreamingMetrics_streaming_lastCompletedBatch_schedulingDelay_value",
        "driver_StreamingMetrics_streaming_lastReceivedBatch_records_value",
    ],
    [
        "driver_StreamingMetrics_streaming_lastCompletedBatch_schedulingDelay_value",
        "driver_StreamingMetrics_streaming_lastReceivedBatch_records_value",
    ],
    [
        "driver_StreamingMetrics_streaming_lastCompletedBatch_schedulingDelay_value",
        "driver_StreamingMetrics_streaming_lastReceivedBatch_records_value",
    ],
]

In [ ]:
def compute_instability(explanations: list):
    """
    Compute the instability of a list of explanations.

    Parameters:
    explanations (list): A list of explanations.

    Returns:
    float: The instability value, which is a measure of how stable the explanations are.
    """
    instability = 0
    flattened_explanations = [item for sublist in explanations for item in sublist]
    unique_explanations = set(flattened_explanations)
    for feature in unique_explanations:
        p = flattened_explanations.count(feature) / len(flattened_explanations)
        instability += -p * np.log2(p)
    # instability = 1 - len(unique_explanations) / len(explanations)

    return instability

In [ ]:
compute_instability(explanations)